In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

Using TensorFlow backend.
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
set_session(tf.Session(config=config))

In [4]:
ft = fastText.load_model("/home1/zishan/raghav/wiki.hi.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [5]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [6]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [7]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [8]:
def train_dev_sentences(filetrain, filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]
    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)

In [9]:
train_file = '/home1/zishan/raghav/Data/train_total.txt'
dev_file = '/home1/zishan/raghav/Data/test_total.txt'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(train_file, dev_file, 3)

In [10]:
n_words = 0
for sentence in train_sentences:
    n_words+=len(sentence)
print(n_words)

239628


In [11]:
from collections import Counter
print(Counter(train_labels))
print(labels2Idx)

Counter({0: 1475, 2: 368, 1: 212, 3: 176, 7: 147, 4: 143, 6: 62, 5: 52, 8: 32})
{'SADNESS': 0, 'NO-EMOTION': 1, 'SYMPATHY/PENSIVENESS': 2, 'OPTIMISM': 3, 'JOY': 4, 'ANGER': 5, 'DISGUST': 6, 'FEAR/ANXIETY': 7, 'SURPRISE': 8}


In [12]:
print(train_sentences[:2])
print(train_labels[:2])
print(labels2Idx)
print(len(train_labels))

['इसके बाद से यहां पर भारतीय सेना तैनात रहती है।', 'सरकारी मीडिया ने सोमवार को बताया कि बाढ़ के भंवर में फंस कर फुजिआन प्रांत में ४३, हुनान में ७८ और गुआंगडांग में ३३ लोगों की मौत हो गई।']
[0, 0]
{'SADNESS': 0, 'NO-EMOTION': 1, 'SYMPATHY/PENSIVENESS': 2, 'OPTIMISM': 3, 'JOY': 4, 'ANGER': 5, 'DISGUST': 6, 'FEAR/ANXIETY': 7, 'SURPRISE': 8}
2667


In [13]:
n_labels = len(labels2Idx)

In [ ]:
def compile_model_bilstm(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)
    model_concatenated = Flatten()(lstm_block)
    model_concatenated = Dense(100)(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

In [14]:
def compile_model_bilstm_cnn(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

# Build Pretraining Model

In [ ]:
model = compile_model_bilstm_cnn(no_labels = n_labels)

In [ ]:
train_file = '/home1/zishan/raghav/Data/train_total.txt'
weights_file ='/home1/zishan/raghav/weights/without_pretraining.h5'
log_file = '/home1/zishan/raghav/logs/without_pretraining.txt'
batch_size = 16
check_for_generator = 3
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [ ]:
max_f1 = 0
for epoch in range(200):
    print("Epoch {}".format(epoch))
    model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx,tokenize= tokenize),
                        steps_per_epoch= steps_per_epoch, epochs=1,)

    testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
    for i in range(len(dev_sentences)):
        testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
    results = model.predict(testset_features)


    predLabels = results.argmax(axis=-1)
    devLabels = dev_labels
    f1 = f1_score(devLabels, predLabels, average='macro') # offensive is the major class. So other is minor
    r = recall_score(devLabels, predLabels, average='macro')
    p = precision_score(devLabels, predLabels, average='macro')
    a = accuracy_score(devLabels, predLabels)
    
    if f1> max_f1:
        print("Saved ")
        model.save_weights(weights_file)
        with open(log_file,'a+') as f:
            text = "{0} , a:{1}, f1:{2}, p:{3}, r:{4}\n".format(epoch, a, f1, p, r)
            f.write(text)
        max_f1 = f1

    print(a,f1)

# transfer learning

In [21]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               transmat: 'matrix if crosslingual training'=None,
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    f.close()
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model_bilstm_cnn(4)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(len(labels2Idx), activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save_weights(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            text = "************\nUnfreezing {}\n****************\n".format(final_model.layers[ulayer[0]].name)
            with open(filename_to_log,'a') as f:
                f.write(text)            
            print(text)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx, tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save_weights(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                    to_write = "p:{0}\t r:{1}\t f1(macro):{2}\t acc:{3}\n".format(max_p,max_r,max_f1,max_a)
                    with open(filename_to_log,'a') as f:
                        f.write(to_write)

                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
                
        to_write = "p:{0}\t r:{1}\t f1(macro):{2}\t acc:{3}\n".format(max_p,max_r,max_f1,max_a)
#         to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        with open(filename_to_log,'a') as f:
            f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    with open(filename_to_log,'a') as f:
        f.write(final_text)

In [22]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (18,16), (18,3), (18,1)]

strings =['suf_bu', 'suf_td','all_unfreeze','gradual_unfreeze']
# strings = ['suf_td','all_unfreeze', 'gradual_unfreeze']
# strings=['gradual_unfreeze']
unfreeze_strategy = [single_unfreeze_bottom_up, single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]
# unfreeze_strategy = [gradual_unfreezing]
# unfreeze_strategy = [ single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]


In [ ]:
for i in range(len(strings)):
    string = strings[i]
    print("approach: %s" %(string))
    
    generator = sequential_generator
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 1
    transmat = None
    number_of_epochs = 200
    labels2Id = labels2Idx
    log_file = '/home1/zishan/raghav/logs/tl_hindi_sentiments_' +string+'.txt' 
    print("log file: %s" %(log_file))
    weights_file='/home1/zishan/raghav/weights/tl_hindi_sentiments_'+string+'.h5'
    print("save weights file: %s" %(weights_file))
    batch_size=16
    train_file='/home1/zishan/raghav/Data/train_total.txt'
    f1_measure='macro'
    pos_label=1
    strategy = unfreeze_strategy[i]
    print(strategy)
    load_model_weights=True
    load_weights_file = '/home1/zishan/raghav/weights/pretrain_bilstm_3cnn_dropout=0.5.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=3
    
    test_model_tl_unfreezing(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=log_file, 
           labels2Idx = labels2Id,
           filename_to_save_weigths=weights_file,
           batch_size=batch_size,
           unfreezing_strategy = strategy,       
           train_file=train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=True,
           model_weights_file = load_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           transmat = transmat,
           check_for_generator= check_for_generator)

approach: suf_bu
log file: /home1/zishan/raghav/logs/tl_hindi_sentiments_suf_bu.txt
save weights file: /home1/zishan/raghav/weights/tl_hindi_sentiments_suf_bu.h5
[(18, 18), (17, 16), (15, 3), (2, 1), (18, 1)]
Test 1/1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, 75, 200)      320800      input_8[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_36 (LeakyReLU)      (None, 75, 200)      0           bidirectional_8[0][0]            
_________________________________________________________________________________________

/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


167/167 [==============================] - 17s 101ms/step - loss: 1.6864 - acc: 0.5127
Test-Data: Prec: 0.176, Rec: 0.123, F1: 0.105, Acc: 0.561
Epoch: 3/200
Epoch 1/1
167/167 [==============================] - 17s 100ms/step - loss: 1.6242 - acc: 0.5206
model saved. F1 is 0.122143
Test-Data: Prec: 0.178, Rec: 0.136, F1: 0.122, Acc: 0.575
Epoch: 4/200
Epoch 1/1
167/167 [==============================] - 17s 101ms/step - loss: 1.5601 - acc: 0.5400
Test-Data: Prec: 0.141, Rec: 0.135, F1: 0.119, Acc: 0.575
Epoch: 5/200
Epoch 1/1
167/167 [==============================] - 17s 100ms/step - loss: 1.5544 - acc: 0.5453
Test-Data: Prec: 0.145, Rec: 0.136, F1: 0.119, Acc: 0.576
Epoch: 6/200
Epoch 1/1
167/167 [==============================] - 17s 101ms/step - loss: 1.5230 - acc: 0.5516
Test-Data: Prec: 0.145, Rec: 0.130, F1: 0.113, Acc: 0.572
Epoch: 7/200
Epoch 1/1
167/167 [==============================] - 17s 100ms/step - loss: 1.5256 - acc: 0.5498
Test-Data: Prec: 0.137, Rec: 0.132, F1: 0.114

167/167 [==============================] - 15s 92ms/step - loss: 1.4280 - acc: 0.5618
Test-Data: Prec: 0.140, Rec: 0.134, F1: 0.117, Acc: 0.575
Epoch: 100/200
Epoch 1/1
167/167 [==============================] - 15s 90ms/step - loss: 1.4361 - acc: 0.5576
Test-Data: Prec: 0.130, Rec: 0.132, F1: 0.113, Acc: 0.572
Epoch: 101/200
Epoch 1/1
167/167 [==============================] - 15s 91ms/step - loss: 1.4319 - acc: 0.5573
Test-Data: Prec: 0.138, Rec: 0.135, F1: 0.119, Acc: 0.575
Epoch: 102/200
Epoch 1/1
167/167 [==============================] - 15s 91ms/step - loss: 1.4321 - acc: 0.5573
Test-Data: Prec: 0.137, Rec: 0.132, F1: 0.114, Acc: 0.572
Epoch: 103/200
Epoch 1/1
167/167 [==============================] - 15s 89ms/step - loss: 1.4322 - acc: 0.5543
Test-Data: Prec: 0.251, Rec: 0.135, F1: 0.119, Acc: 0.576
Epoch: 104/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.4313 - acc: 0.5625
Test-Data: Prec: 0.140, Rec: 0.131, F1: 0.112, Acc: 0.573
Epoch: 105/

Test-Data: Prec: 0.249, Rec: 0.138, F1: 0.124, Acc: 0.575
Epoch: 148/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.4440 - acc: 0.5621
Test-Data: Prec: 0.131, Rec: 0.130, F1: 0.111, Acc: 0.572
Epoch: 149/200
Epoch 1/1
167/167 [==============================] - 15s 93ms/step - loss: 1.4279 - acc: 0.5573
Test-Data: Prec: 0.249, Rec: 0.135, F1: 0.120, Acc: 0.575
Epoch: 150/200
Epoch 1/1
167/167 [==============================] - 15s 90ms/step - loss: 1.4325 - acc: 0.5550
Test-Data: Prec: 0.134, Rec: 0.132, F1: 0.114, Acc: 0.572
Epoch: 151/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.4335 - acc: 0.5576
Test-Data: Prec: 0.134, Rec: 0.131, F1: 0.114, Acc: 0.570
Epoch: 152/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.4243 - acc: 0.5550
Test-Data: Prec: 0.134, Rec: 0.132, F1: 0.114, Acc: 0.572
Epoch: 153/200
Epoch 1/1
167/167 [==============================] - 15s 93ms/step - loss: 1.4258 

...

Test-Data: Prec: 0.251, Rec: 0.139, F1: 0.123, Acc: 0.579
Epoch: 160/200
Epoch 1/1
167/167 [==============================] - 15s 91ms/step - loss: 1.4392 - acc: 0.5618
Test-Data: Prec: 0.139, Rec: 0.137, F1: 0.121, Acc: 0.576
Epoch: 161/200
Epoch 1/1
167/167 [==============================] - 15s 91ms/step - loss: 1.4257 - acc: 0.5576
Test-Data: Prec: 0.249, Rec: 0.133, F1: 0.117, Acc: 0.573
Epoch: 162/200
Epoch 1/1
167/167 [==============================] - 15s 91ms/step - loss: 1.4270 - acc: 0.5550
Test-Data: Prec: 0.134, Rec: 0.132, F1: 0.114, Acc: 0.572
Epoch: 163/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.4179 - acc: 0.5580
Test-Data: Prec: 0.253, Rec: 0.133, F1: 0.117, Acc: 0.573
Epoch: 164/200
Epoch 1/1
167/167 [==============================] - 15s 90ms/step - loss: 1.4550 - acc: 0.5550
Test-Data: Prec: 0.142, Rec: 0.132, F1: 0.115, Acc: 0.573
Epoch: 165/200
Epoch 1/1
167/167 [==============================] - 15s 91ms/step - loss: 1.4369 

167/167 [==============================] - 43s 258ms/step - loss: 1.2952 - acc: 0.5838
model saved. F1 is 0.144152
Test-Data: Prec: 0.282, Rec: 0.151, F1: 0.144, Acc: 0.584
Epoch: 6/200
Epoch 1/1
167/167 [==============================] - 43s 256ms/step - loss: 1.2837 - acc: 0.5786
model saved. F1 is 0.158808
Test-Data: Prec: 0.372, Rec: 0.160, F1: 0.159, Acc: 0.585
Epoch: 7/200
Epoch 1/1
167/167 [==============================] - 42s 252ms/step - loss: 1.2628 - acc: 0.5846
model saved. F1 is 0.160558
Test-Data: Prec: 0.361, Rec: 0.161, F1: 0.161, Acc: 0.588
Epoch: 8/200
Epoch 1/1
167/167 [==============================] - 42s 254ms/step - loss: 1.2437 - acc: 0.5879
model saved. F1 is 0.192308
Test-Data: Prec: 0.432, Rec: 0.179, F1: 0.192, Acc: 0.588
Epoch: 9/200
Epoch 1/1
167/167 [==============================] - 44s 264ms/step - loss: 1.2218 - acc: 0.5909
model saved. F1 is 0.195986
Test-Data: Prec: 0.423, Rec: 0.184, F1: 0.196, Acc: 0.588
Epoch: 10/200
Epoch 1/1
167/167 [==========

167/167 [==============================] - 44s 265ms/step - loss: 0.7805 - acc: 0.7275
Test-Data: Prec: 0.493, Rec: 0.295, F1: 0.324, Acc: 0.629
Epoch: 51/200
Epoch 1/1
167/167 [==============================] - 44s 266ms/step - loss: 0.7786 - acc: 0.7279
model saved. F1 is 0.337790
Test-Data: Prec: 0.507, Rec: 0.305, F1: 0.338, Acc: 0.636
Epoch: 52/200
Epoch 1/1
167/167 [==============================] - 45s 267ms/step - loss: 0.8130 - acc: 0.7242
model saved. F1 is 0.344596
Test-Data: Prec: 0.508, Rec: 0.313, F1: 0.345, Acc: 0.638
Epoch: 53/200
Epoch 1/1
167/167 [==============================] - 44s 264ms/step - loss: 0.7742 - acc: 0.7283
model saved. F1 is 0.345734
Test-Data: Prec: 0.518, Rec: 0.308, F1: 0.346, Acc: 0.639
Epoch: 54/200
Epoch 1/1
167/167 [==============================] - 44s 263ms/step - loss: 0.7586 - acc: 0.7317
Test-Data: Prec: 0.497, Rec: 0.286, F1: 0.325, Acc: 0.629
Epoch: 55/200
Epoch 1/1
167/167 [==============================] - 44s 265ms/step - loss: 0.743

167/167 [==============================] - 44s 265ms/step - loss: 0.5578 - acc: 0.8185
model saved. F1 is 0.398868
Test-Data: Prec: 0.501, Rec: 0.363, F1: 0.399, Acc: 0.645
Epoch: 98/200
Epoch 1/1
167/167 [==============================] - 44s 263ms/step - loss: 0.5605 - acc: 0.8080
model saved. F1 is 0.408160
Test-Data: Prec: 0.517, Rec: 0.366, F1: 0.408, Acc: 0.653
Epoch: 99/200
Epoch 1/1
167/167 [==============================] - 44s 263ms/step - loss: 0.5357 - acc: 0.8129
Test-Data: Prec: 0.478, Rec: 0.342, F1: 0.382, Acc: 0.648
Epoch: 100/200
Epoch 1/1
167/167 [==============================] - 44s 264ms/step - loss: 0.5538 - acc: 0.8118
Test-Data: Prec: 0.463, Rec: 0.336, F1: 0.374, Acc: 0.641
Epoch: 101/200
Epoch 1/1
167/167 [==============================] - 44s 265ms/step - loss: 0.5368 - acc: 0.8125
Test-Data: Prec: 0.473, Rec: 0.344, F1: 0.383, Acc: 0.647
Epoch: 102/200
Epoch 1/1
167/167 [==============================] - 44s 262ms/step - loss: 0.5394 - acc: 0.8125
Test-Data

167/167 [==============================] - 44s 262ms/step - loss: 0.4156 - acc: 0.8597
Test-Data: Prec: 0.467, Rec: 0.324, F1: 0.365, Acc: 0.648
Epoch: 146/200
Epoch 1/1
167/167 [==============================] - 44s 263ms/step - loss: 0.4172 - acc: 0.8649
Test-Data: Prec: 0.498, Rec: 0.353, F1: 0.397, Acc: 0.671
Epoch: 147/200
Epoch 1/1
167/167 [==============================] - 44s 261ms/step - loss: 0.4148 - acc: 0.8660
Test-Data: Prec: 0.473, Rec: 0.344, F1: 0.387, Acc: 0.659
Epoch: 148/200
Epoch 1/1
167/167 [==============================] - 44s 262ms/step - loss: 0.4212 - acc: 0.8619
Test-Data: Prec: 0.463, Rec: 0.344, F1: 0.384, Acc: 0.660
Epoch: 149/200
Epoch 1/1
167/167 [==============================] - 44s 261ms/step - loss: 0.4058 - acc: 0.8713
Test-Data: Prec: 0.475, Rec: 0.338, F1: 0.381, Acc: 0.654
Epoch: 150/200
Epoch 1/1
167/167 [==============================] - 44s 264ms/step - loss: 0.4240 - acc: 0.8589
Test-Data: Prec: 0.473, Rec: 0.331, F1: 0.375, Acc: 0.660
Epoch

In [26]:
print(Counter(dev_labels))

Counter({0: 374, 2: 93, 1: 61, 3: 43, 4: 37, 7: 29, 6: 16, 8: 9, 5: 6})
